In [1]:
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df_map = pd.read_csv('/content/drive/MyDrive/DA ASSIGNMENT/DATA/divvy_final.csv',parse_dates=['start_time', 'end_time'])

<ipython-input-2-757875827277>:1: DtypeWarning: Columns (4,6,8,9,10,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_map = pd.read_csv('/content/drive/MyDrive/DA ASSIGNMENT/DATA/divvy_final.csv',parse_dates=['start_time', 'end_time'])


In [3]:
df_map.head(3)

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,Quarter,station_name_x,from_station_latitude,from_station_longitude,station_name_y,to_station_latitude,to_station_longitude
0,13518905.0,2017-03-31 23:59:07,2017-04-01 00:13:24,5292.0,857,66.0,Clinton St & Lake St,171.0,May St & Cullerton St,Subscriber,Male,1989.0,Q1,Clinton St & Lake St,41.885637,-87.641823,May St & Cullerton St,41.855136,-87.654127
1,13518904.0,2017-03-31 23:56:25,2017-04-01 00:00:21,4408.0,236,199.0,Wabash Ave & Grand Ave,26.0,McClurg Ct & Illinois St,Subscriber,Male,1990.0,Q1,Wabash Ave & Grand Ave,41.890912,-87.626804,McClurg Ct & Illinois St,41.891020,-87.617300
2,13518903.0,2017-03-31 23:55:33,2017-04-01 00:01:21,696.0,348,520.0,Greenview Ave & Jarvis Ave,432.0,Clark St & Lunt Ave,Subscriber,Female,1979.0,Q1,Greenview Ave & Jarvis Ave,42.015962,-87.668570,Clark St & Lunt Ave,42.009074,-87.674190


In [4]:
df_map['startdate_time'] = pd.to_datetime(df_map['start_time'], errors='coerce')

In [5]:
df_map['dayoftheWeek'] = df_map['startdate_time'].dt.day_name()
df_map['Month'] = df_map['startdate_time'].dt.month_name()
df_map['Year'] = df_map['startdate_time'].dt.year
df_map['StartingTime']=df_map['startdate_time'].dt.time

In [6]:
df_map['dayoftheWeek'] = pd.Categorical(df_map['dayoftheWeek'], categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'],ordered=True)

week1=(df_map['dayoftheWeek'].value_counts()*100)/len(df_map)
week2 = week1.sort_index()

In [7]:
df_map['Month'] = pd.Categorical(df_map['Month'], categories= ['January', 'February', 'March', 'April', 'May', 'June','July', 'August', 'September', 'October', 'November', 'December'],ordered=True)
month=df_map['Month'].value_counts()/1000
month = month.sort_index()

In [8]:
def identify_season(Month):
    season = None
    if Month == 'January' or Month == 'February' or Month == 'December':
        season = 'Winter'
    elif Month == 'March' or Month == 'April' or Month == 'May':
        season = 'Spring'
    elif Month == 'June' or Month == 'July' or Month == 'August':
        season = 'Summer'
    elif Month == 'September' or Month == 'October' or Month == 'November':
        season = 'Autumn'
    return season

In [9]:
season_list = []
for Month in df_map['Month']:
  season_list.append(identify_season(Month))
df_map['season'] = season_list

In [10]:
df_map['season'].unique()

array(['Spring', 'Winter', 'Summer', 'Autumn', None], dtype=object)

In [11]:
df_map = df_map.dropna(subset=['gender'])

In [12]:
df_map['tripduration'] = df_map['tripduration'].str.replace(',', '').astype(float)

In [13]:
import plotly.express as px
fig = px.sunburst(df_map,
                  path=['gender','Month','dayoftheWeek'],
                  values ='tripduration',
                  color = 'tripduration',
                  title = 'Trips taken by Males vs Females'
                   )
fig.update_layout(
    autosize=False,
    height=700,
    width=700)

fig.show()

In [14]:
import folium
from folium.plugins import HeatMap
lat_lon_pair=list(zip(list(df_map.from_station_latitude),list(df_map.from_station_longitude)))

In [15]:
map_df = df_map[['from_station_name','from_station_latitude','from_station_longitude']].drop_duplicates(subset='from_station_name')

In [16]:
df_map.isnull().sum()

trip_id                         0
start_time                      0
end_time                        0
bikeid                          0
tripduration              2992187
from_station_id                 0
from_station_name               0
to_station_id                   0
to_station_name                 0
usertype                        0
gender                          0
birthyear                     304
Quarter                         0
station_name_x             412308
from_station_latitude      412308
from_station_longitude     412308
station_name_y             408354
to_station_latitude        408354
to_station_longitude       408354
startdate_time                  0
dayoftheWeek                    0
Month                           0
Year                            0
StartingTime                    0
season                          0
dtype: int64

In [17]:
df_map.dropna(subset=['from_station_latitude','from_station_longitude','to_station_latitude','to_station_longitude'], inplace=True)

In [18]:
import folium

# Assuming map_df is your DataFrame
map_df_cleaned = map_df.dropna(subset=['from_station_latitude', 'from_station_longitude'])

# Create the map
map_at = folium.Map(location=[48.3794, -87.623177], zoom_start=13, center=[41.8781136, -87.623177])

# Add markers to the map
for index, row in map_df_cleaned.iterrows():
    folium.Marker(location=[row["from_station_latitude"], row["from_station_longitude"]],
                  popup=row['from_station_name']).add_to(map_at)

# Display the map
map_at

In [19]:
import folium
from folium.plugins import HeatMap

# Assuming lat_lon_pair is your list of coordinates (latitude, longitude)
lat_lon_pair_cleaned = [pair for pair in lat_lon_pair if all(isinstance(coord, (float, int)) and not pd.isna(coord) for coord in pair)]

# Create the map
map_heat = folium.Map(location=[41.8781136, -87.623177], zoom_start=12)

# Add HeatMap to the map
HeatMap(lat_lon_pair_cleaned[0:25000]).add_to(map_heat)

# Display the map
map_heat

In [20]:
route =df_map.groupby(['from_station_name','to_station_name']).size().sort_values(ascending=False).reset_index(name='count').drop_duplicates(subset="to_station_name")
route['trip'] = route['from_station_name']+ ' to '+ route['to_station_name']

In [21]:
from folium import plugins
most_popular_route = folium.Map(location=[41.880958,-87.616743],zoom_start=15)
loc = [[41.879255,-87.639904],[41.88389277,-87.62464914]]
plugins.AntPath(loc).add_to(most_popular_route)
folium.PolyLine(loc,
                color='red',
                weight=15,
                opacity=0.5).add_to(most_popular_route)
most_popular_route

In [22]:
round_df = (df_map[df_map['from_station_name'] == df_map['to_station_name']])
count_of_roundtrip = len(round_df.index)
print("Total number of round trip :", count_of_roundtrip)

Total number of round trip : 145300


In [23]:
round_trip_per = (round(count_of_roundtrip*100/len(df_map),2))
print('Percentage of round trip is {} %.'.format(round_trip_per))

Percentage of round trip is 2.01 %.


In [24]:
to_less_popular_station_df  = df_map['from_station_name'].value_counts(ascending= False).tail(10).reset_index(name='counts')
to_less_popular_station_df .columns = ['from_station_name','Counts']
to_less_popular_station_df .head(5)

,from_station_name,Counts
0,Normal Ave & 72nd St,49
1,Cicero Ave & Flournoy St,42
2,Vernon Ave & 79th St,42
3,Throop St & 52nd St,40
4,South Chicago Ave & 83rd St,39


In [25]:
less_popular_station_df = df_map['from_station_name'].value_counts(ascending= False).tail(10).reset_index(name='counts')
less_popular_station_df .columns = ['from_station_name','Counts']
fig = px.bar(less_popular_station_df ,x='from_station_name', y ='Counts',text_auto =True, title='Least Popular Divvy Bike Stations')
fig.update_layout(xaxis_title ='Station',yaxis_title = 'Trips Originating from these Station')
fig.show()

In [26]:
df_map["dayoftheWeek"].value_counts()

Tuesday      1198069
Wednesday    1175878
Thursday     1172770
Monday       1115568
Friday       1089324
Saturday      763431
Sunday        704889
Name: dayoftheWeek, dtype: int64

In [27]:
df_map["Month"].value_counts()

August       1179619
July         1129400
September    1024679
October       828014
June          697304
May           546928
November      443512
April         365869
December      335968
March         264705
February      217260
January       186671
Name: Month, dtype: int64

In [28]:
df_map["Quarter"].value_counts()

Q3     1133246
Q11    1101293
Q7     1099159
Q2      822547
Q6      787554
Q4      572621
Q8      517821
Q12     517052
Q1      376661
Q9      291975
Name: Quarter, dtype: int64

In [29]:
trip_duration = df_map.groupby(['season'])['tripduration'].mean().sort_values(ascending=False)
trip_duration

season
Summer    1142.421763
Autumn     959.818604
Spring     835.168885
Winter     823.810802
Name: tripduration, dtype: float64

In [30]:
df_map.describe()

,trip_id,bikeid,tripduration,from_station_id,to_station_id,birthyear,from_station_latitude,from_station_longitude,to_station_latitude,to_station_longitude,Year
count,7.219929e+06,7.219929e+06,4.314854e+06,7.219929e+06,7.219929e+06,7.219735e+06,7.219929e+06,7.219929e+06,7.219929e+06,7.219929e+06,7.219929e+06
mean,1.921655e+07,3.445539e+03,1.009888e+03,1.872170e+02,1.879177e+02,1.982732e+03,4.189962e+01,-8.764381e+01,4.190021e+01,-8.764415e+01,2.017862e+03
std,3.850227e+06,1.901423e+03,1.846679e+04,1.319279e+02,1.321842e+02,1.097149e+01,3.769937e-02,2.255084e-02,3.801840e-02,2.264229e-02,8.049694e-01
min,1.297923e+07,1.000000e+00,6.100000e+01,2.000000e+00,2.000000e+00,1.889000e+03,4.173665e+01,-8.780287e+01,4.173665e+01,-8.780287e+01,2.017000e+03
25%,1.589137e+07,1.785000e+03,3.730000e+02,7.700000e+01,7.700000e+01,1.977000e+03,4.188096e+01,-8.765453e+01,4.188103e+01,-8.765479e+01,2.017000e+03
50%,1.911183e+07,3.571000e+03,6.170000e+02,1.660000e+02,1.660000e+02,1.986000e+03,4.189228e+01,-8.763990e+01,4.189388e+01,-8.764083e+01,2.018000e+03
75%,2.189550e+07,5.124000e+03,1.046000e+03,2.830000e+02,2.830000e+02,1.991000e+03,4.191994e+01,-8.762952e+01,4.192154e+01,-8.762952e+01,2.019000e+03
max,2.596290e+07,6.946000e+03,1.146910e+07,6.230000e+02,6.230000e+02,2.017000e+03,4.206400e+01,-8.754939e+01,4.206400e+01,-8.754939e+01,2.019000e+03


In [31]:
df_map.duplicated().sum()

0